<a href="https://colab.research.google.com/github/gt0420-bit/GT-Project/blob/main/Copy_of_CrewAI_agent_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CrewAI demo notebook: Research a topic, generate content and get social media ready post and a blog article!

# 📦 Install/Upgrade Dependencies
This cell installs the libraries used in the workflow:
- `openai` (new SDK), `transformers` for model helpers
- `crewai`, `langchain`, `langchain-openai` for agent orchestration
- `crewai-tools` for optional tool helpers
- `requests` for webhooks / HTTP calls

> Tip: After upgrading key packages in Colab, use **Runtime → Restart session** to ensure fresh imports.


In [ ]:
# !pip install crewai openai requests langchain
!pip install --upgrade openai
!pip install transformers -U
!pip install -U "crewai>=0.56" "langchain>=0.2.11" "langchain-core>=0.2.11" \
              "langchain-openai>=0.1.14" requests
!pip install crewai-tools
!pip install transformers -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.8/999.8 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 138.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.3 MB/s 

# 🔐 Core Imports & OpenAI API Key
- Imports common libs and CrewAI/LangChain wrappers.
- Reads your OpenAI key (e.g., from Colab `userdata` or a variable you set).
- Exposes `OPENAI_API_KEY` to the environment.
- Creates a `ChatOpenAI` client for quick single-shot prompts if needed.

> Ensure `openai_api_key` is defined (e.g., `from google.colab import userdata; openai_api_key = userdata.get('OPENAI_API_KEY')`)


In [ ]:
# pip install crewai openai requests
import os
import numpy as np
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI # Updated import
import base64, os, requests
from google.colab import userdata # Import userdata
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool
# from diffusers import StableDiffusionPipeline
import torch
from typing import Annotated # Import Annotated
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=os.environ["OPENAI_API_KEY"])  # Pass API key here

# 🧩 Helper Functions (Social Copy + Webhook)
- `_make_social_copy_impl(title, summary, platform)` calls an LLM to craft a short post tailored to a platform (LinkedIn/Twitter).
- `_post_via_webhook_impl(text, image_path)` sends text (and optional image) to an external webhook URL from `SOCIAL_WEBHOOK_URL`.

> ⚠️ Set `SOCIAL_WEBHOOK_URL` in your environment if you plan to publish.


# 🧱 Tool Base Class Import (Version-safe)
- Tries `crewai.tools.BaseTool`, then `crewai_tools.BaseTool`, then a minimal fallback class.
- This keeps the notebook working across CrewAI versions.


# 🛠️ Class-Based Tools for CrewAI
- `MakeSocialCopyTool`: wraps `_make_social_copy_impl`.
- `PostViaWebhookTool`: wraps `_post_via_webhook_impl`.

These tools can be attached to agents so tasks can call them.


In [ ]:
import os, requests
from typing import Optional, Type
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [ ]:
from crewai.tools import BaseTool

In [ ]:
def _make_social_copy_impl(title: str, summary: str, platform: str = "linkedin") -> str:
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = (
        f"Craft a concise {platform} post promoting an article.\n"
        f"Title: {title}\nSummary: {summary}\n"
        "Constraints: 1-2 punchy lines, clear CTA, 3-5 relevant hashtags."
    )
    return llm.invoke(prompt).content

def _post_via_webhook_impl(text: str, image_path: Optional[str] = None) -> str:
    WEBHOOK_URL = os.environ["SOCIAL_WEBHOOK_URL"]
    files = {"image": open(image_path, "rb")} if image_path else None
    r = requests.post(WEBHOOK_URL, data={"text": text}, files=files, timeout=60)
    r.raise_for_status()
    return "Posted"

class MakeSocialCopyTool(BaseTool):
    name: str = "make_social_copy"
    description: str = "Create a short, platform-optimized social post from a title and summary. Args: title (str), summary (str), platform (str, optional)"

    def _run(self, title: str, summary: str, platform: str = "linkedin") -> str:
        return _make_social_copy_impl(title, summary, platform)

class PostViaWebhookTool(BaseTool):
    name: str = "post_via_webhook"
    description: str = "Publish a post (and optional image) via a webhook integration. Args: text (str), image_path (str, optional)"

    def _run(self, text: str, image_path: Optional[str] = None) -> str:
        return _post_via_webhook_impl(text, image_path)

# Create tool instances
# generate_image_local = GenerateImageTool()
make_social_copy = MakeSocialCopyTool()
post_via_webhook = PostViaWebhookTool()


# 🔁 Function-Based Tool Fallbacks
- Supplies plain functions equivalent to the class-based tools.
- Some CrewAI versions prefer function tools; this keeps things compatible.


# 👥 Agents: Researcher, Writer, Social Strategist
- **Researcher**: gathers 5 current insights (no web tool here; relies on LLM knowledge unless you add a search tool).
- **Writer**: converts notes into a short article.
- **Social Media Strategist**: uses the tools to create platform-specific posts, optionally publish via webhook.
- Includes a try/except to attach class-based tools first, then fall back to function-based tools.


In [ ]:
# ---------- Agents ----------
from crewai import Agent
from langchain_openai import ChatOpenAI

# Make sure to set your OpenAI API key
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

researcher = Agent(
    role="Researcher",
    goal="Collect 5 up-to-date, credible insights on the topic.",
    backstory="Finds and synthesizes sources.",
    allow_delegation=False,
    llm=llm,
)

writer = Agent(
    role="Writer",
    goal="Draft a 600-900 word article from notes with clear structure.",
    backstory="Senior technical writer.",
    allow_delegation=False,
    llm=llm,
)

# Try using the class-based tools first, fallback to functions if needed
# try:

social_agent = Agent(
    role="Social Media Strategist",
    goal="Adapt article into a platform-optimized post and (optionally) publish.",
    backstory="Knows tone/length/hashtag best practices.",
    tools=[make_social_copy, post_via_webhook],
    allow_delegation=False,
    llm=llm,
)
print("Successfully created agents with class-based tools")
print("All agents created successfully!")

Successfully created agents with class-based tools
All agents created successfully!


# 📝 Tasks & Context Flow
- `t1` (Researcher): produce 5 concise insights with short citations.
- `t2` (Writer): create a ~250–300 word article from `t1`'s output (title, summary, sections, conclusion).
- `t3` (Social): extract title/summary from the article (context=`t2`), produce LinkedIn + Twitter versions, **don’t publish** (just show drafts).

> `context=[t1]` or `[t2]` passes upstream outputs downstream.


In [ ]:
# ---------- Create Tasks ----------
t1 = Task(
    description="Research the topic: 'State of Canada's economic growth in the 21st century'. Find 5 key insights with current developments, challenges, and opportunities. Include brief references to sources.",
    agent=researcher,
    expected_output="Bulleted list of 5 insights with short citations or source references.",
)

t2 = Task(
    description="Write a structured 250 to 300 word article from the research notes. Include: compelling title, introduction, 3-4 main sections, and conclusion. Start with a brief summary paragraph.",
    agent=writer,
    expected_output="Complete markdown article with title, summary paragraph, and well-structured sections.",
    context=[t1],
)


t3 = Task(
    description="""Create social media posts for the article:
    1. Use make_social_copy to create a LinkedIn version (title from article, summary from article, platform='linkedin')
    2. Use make_social_copy to create an X/Twitter version (title from article, summary from article, platform='twitter')
    3. For testing purposes, do NOT publish automatically - just show what the posts would look like

    Extract the title and summary from the article context to use with the tools.""",
    agent=social_agent,
    expected_output="LinkedIn post version, X/Twitter post version, and publishing status.",
    context=[t1, t2],
)

# 🚀 Build Crew & Run
- Creates the `Crew` with your agents and tasks.
- `verbose=True` to see execution traces.
- Calls `crew.kickoff()` to execute the workflow end-to-end.
- Catches and prints any runtime errors (e.g., bad API key, rate limit, missing webhook).


In [ ]:
crew = Crew(
    agents=[researcher, writer, social_agent],
    tasks=[t1, t2, t3],
    verbose=True  # Set to True or 2 for detailed output
)

print("🚀 Starting CrewAI workflow...")
print("=" * 50)

try:
    result = crew.kickoff()
    print("\n" + "=" * 50)
    print("✅ WORKFLOW COMPLETED!")
    print("=" * 50)
    print(result)
except Exception as e:
    print(f"❌ Error running crew: {e}")
    print("Check your API keys and environment setup.")

🚀 Starting CrewAI workflow...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5b46e3a2-8f32-40cc-8750-6315658d5295                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Task: Research the topic: 'State of Canada's economic growth in the 21st century'. Find 5 key insights with    │
│  current developments, challenges, and opportunities. Include brief references to sources.                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - **Steady Growth Since 2000**: Canada's economy has shown resilience and steady growth since the early        │
│  2000s, with real GDP increasing at an average annual rate of around 2.0%. The recovery post-2008 financial     │
│  crisis led to incremental growth, supported by resource management and exports (Bank of Canada, 2023).         │
│                                                                                                                 │
│  - **Impact of COVID-19**: The pandemic significantly disrupted Canada's economic landscape, resulting in a     │
│  contraction of approximately 5.3% in 2020. However, the subsequent recovery has been robust, with projections  │
│  indicating growth rates of about 3.4% for 2022 and 2.5% for 2023 as public health measures ease and consumer   │
│  spending rebounds (Statistics Canada, 2023).                                                                   │
│                                                                                                                 │
│  - **Challenges of Inflation**: In 2022 and into 2023, Canada has faced heightened inflationary pressures,      │
│  with rates reaching levels not seen in decades. Various factors, including supply chain disruptions and        │
│  rising energy prices, have contributed. The Bank of Canada has responded by increasing interest rates to       │
│  combat inflation, posing potential risks to economic growth (Economist, 2023).                                 │
│                                                                                                                 │
│  - **Opportunities in Green Technology**: Canada is increasingly focusing on green technology and sustainable   │
│  practices as a means to drive future economic growth. The government’s commitment to achieving net-zero        │
│  emissions by 2050 is presenting opportunities for innovation and investment in renewable energy sectors        │
│  (Natural Resources Canada, 2023).                                                                              │
│                                                                                                                 │
│  - **Labor Market Challenges**: The labor market in Canada is currently experiencing challenges, including      │
│  talent shortages in various sectors, especially technology and healthcare. High vacancy rates are impacting    │
│  businesses' ability to grow, highlighting the need for targeted immigration policies and training programs to  │
│  fill these gaps (Conference Board of Canada, 2023).                                                            │
│                                                                                                                 │
│  References:                                                                                                    │
│  - Bank of Canada. (2023). Economic Developments.                                                               │
│  - Statistics Canada. (2023). National Economic Trends.                                                         │
│  - Economist. (2023). Canada’s Inflation Crisis Explained.                                                      │
│  - Natural Resources Canada. (2023). Environmental and Energy Innovation.                                       │
│  - Conference Board of Canada. (2023). Labor Market Ins

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5b46e3a2-8f32-40cc-8750-6315658d5295                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: LinkedIn Post Version:                                                                           │
│  🔍 Dive into the complexities of Canada’s economic journey! Discover the resilience, challenges, and           │
│  opportunities our nation faces as we adapt to a changing landscape. 🇨🇦✨                                       │
│                                                                                                                 │
│  Read more in the full article: [Insert link]                                                                   │
│                                                                                                                 │
│  #CanadaEconomy #Sustainability #EconomicGrowth #JobMarket #FutureOfWork                                        │
│                                                                                                                 │
│  X/Twitter Post Version:                                                                                        │
│  🌍🇨🇦 Dive into Canada’s economic landscape! From resilience to sustainability, discover the challenges and     │
│  opportunities that lie ahead in our latest article. 📈✨ Read more here: [link] #CanadaEconomy                 │
│  #SustainableGrowth #EconomicChallenges #Opportunities #COVID19                                                 │
│                                                                                                                 │
│  Publishing Status: Not published automatically, just showing what the posts would look like.                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): y


╭───────────────────────────────────────── 🔍 Execution Trace Generated ──────────────────────────────────────────╮
│                                                                                                                 │
│  🎉 Your First CrewAI Execution Trace is Ready!                                                                 │
│                                                                                                                 │
│  View your execution details here:                                                                              │
│  https://app.crewai.com/crewai_plus/ephemeral_trace_batches/4f332851-70f2-4e8f-968d-51656c8635c0?access_code=T  │
│  RACE-8f87cadae4                                                                                                │
│                                                                                                                 │
│  This trace shows:                                                                                              │
│  • Agent decisions and interactions                                                                             │
│  • Task execution timeline                                                                                      │
│  • Tool usage and results                                                                                       │
│  • LLM calls and responses                                                                                      │
│                                                                                                                 │
│  ✅ Tracing has been enabled for future runs! (CREWAI_TRACING_ENABLED=true added to .env)                       │
│  You can also add tracing=True to your Crew(tracing=True) / Flow(tracing=True) for more control.                │
│                                                                                                                 │
│  📝 Note: This link will expire in 24 hours.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ WORKFLOW COMPLETED!
LinkedIn Post Version: 
🔍 Dive into the complexities of Canada’s economic journey! Discover the resilience, challenges, and opportunities our nation faces as we adapt to a changing landscape. 🇨🇦✨ 

Read more in the full article: [Insert link] 

#CanadaEconomy #Sustainability #EconomicGrowth #JobMarket #FutureOfWork

X/Twitter Post Version: 
🌍🇨🇦 Dive into Canada’s economic landscape! From resilience to sustainability, discover the challenges and opportunities that lie ahead in our latest article. 📈✨ Read more here: [link] #CanadaEconomy #SustainableGrowth #EconomicChallenges #Opportunities #COVID19

Publishing Status: Not published automatically, just showing what the posts would look like.
